In [1]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import DataLoader
from dataset import LiverTumorDataset

In [3]:
train_dataset = LiverTumorDataset('train_balanced.txt', image_size=(256, 256))
val_dataset   = LiverTumorDataset('val_balanced.txt', image_size=(256, 256))

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=0)

for images, masks in train_loader:
    print("Image batch shape:", images.shape)
    print("Mask batch shape :", masks.shape)
    break

Image batch shape: torch.Size([2, 1, 256, 256])
Mask batch shape : torch.Size([2, 1, 256, 256])


In [7]:
output_dir_tumor = "image_mask_samples/tumor"
output_dir_non_tumor = "image_mask_samples/non_tumor"
os.makedirs(output_dir_tumor, exist_ok=True)
os.makedirs(output_dir_non_tumor, exist_ok=True)

tumor_saved = 0
non_tumor_saved = 0
i = 0
while tumor_saved < 10 or non_tumor_saved < 10:
    image, mask = train_dataset[i]

    image_np = (image.numpy().squeeze() * 255).astype(np.uint8)
    mask_np  = (mask.numpy().squeeze() * 255).astype(np.uint8)

    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_GRAY2BGR)
    mask_bgr = cv2.cvtColor(mask_np, cv2.COLOR_GRAY2BGR)

    is_tumor = np.any(mask_np)

    if is_tumor and tumor_saved < 10:
        cv2.putText(image_bgr, "Tumor Image", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
        cv2.putText(mask_bgr, "Tumor Mask", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
        combined = cv2.hconcat([image_bgr, mask_bgr])
        cv2.imwrite(os.path.join(output_dir_tumor, f"tumor_pair_{tumor_saved}.png"), combined)
        tumor_saved += 1

    elif not is_tumor and non_tumor_saved < 10:
        cv2.putText(image_bgr, "Non-Tumor Image", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
        cv2.putText(mask_bgr, "Empty Mask", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
        combined = cv2.hconcat([image_bgr, mask_bgr])
        cv2.imwrite(os.path.join(output_dir_non_tumor, f"non_tumor_pair_{non_tumor_saved}.png"), combined)
        non_tumor_saved += 1

    i += 1

print("Saved samples to 'image_mask_samples/tumor' and 'non_tumor'")

Saved samples to 'image_mask_samples/tumor' and 'non_tumor'
